In [1]:
from xspec import *
import numpy as np
import os
from scipy.optimize import fsolve
import random

In [ ]:
insts = ["FPMA", "FPMB"]
logFs = [-13] 
nh_gal = 0.0338 
z = 0.001448
exp = [3600, 3600, 2490, 2490, 2490] 
Fit.statMethod = 'cstat'
save_path = '/uufs/chpc.utah.edu/common/home/rott-group3/syu/xray_sim'

In [3]:
def calculate_flux(logNH, logF, i_norm=10):
    def flux_func(x, F=10**logF):
        if x < 0:
            x = 1.0e-10
        AllModels(1)(i_norm).values = x
        AllModels.calcFlux("3.0 24.0")
        return AllModels(1).flux[0] - F

    norm = fsolve(flux_func, 1.0e-5)[0]
    return norm

In [4]:
# Set seed
base_seed = int(1e6)

def parameter_generator(trials, start_ind=0):
    dts=[]
    for trial_id in range(start_ind, trials+start_ind):
        trial_seed = base_seed + trial_id
        random.seed(trial_seed)

        torsigma = random.uniform(7, 84)  # TORsigma
        ctkcover = random.uniform(0.1, 0.6)  # CTKcover
        nh = random.uniform(24, 25)  # nH
        theta_inc = random.uniform(60, 80)  # inclination angle
        phoindex = random.uniform(2.0,2.5)  # Photon index
        #z = random.uniform(0.001,0.2)  # Redshift

        dts.append(dict({
            "trial_id": trial_id,
            "torsigma": torsigma,
            "ctkcover": ctkcover,
            "nh": nh,
            "theta_inc": theta_inc,
            "phoindex": phoindex,
            #"z": z
        }))
    return dts

In [5]:
def set_pars(m,pars,par_vals,
        par_fixed,one_epoch,
        par_var):
    """
    par_vals: the initial values for the parameters
    par_fixed: is the parameter fixed to the initial value?
    par_var: (if multiple_epochs are loaded) are pars linked across epochs?
    """
    n_eps = AllData.nGroups
    for i_par,par in enumerate(pars):
        #If the par is fixed, fix it
        if par_fixed[i_par]:
            m(par).values = [par_vals[i_par],-1]
        #else, if the par is free, free it
        else:
            m(par).values = par_vals[i_par]
            #THEN go through and unlink the parameters (if applicable)
            if not one_epoch and par_var[i_par]:
                for i_ep in range(1,n_eps+1):
                    #unlink the parameter, set it to the starter value
                    AllModels(i_ep)(par).values = par_vals[i_par]

In [12]:
def mk_clumpy(nh_gal, z, random_params=None,
              pars=[1, 3, 4, 10, 11, 20, 25],
              par_vals=[1.0, 1, 1.8, 1.0e-4, 1.0e-3, 0.6, 1.0e-4],
              par_fixed=[True, False, True, False, False, True, False],
              one_epoch=True, 
              par_var=[False, False, False, False, False, False, False],
              bxa=False):
    """
    Define a clumpy torus model with options for dynamic parameter adjustment.
    """
    Xset.abund = "wilm"  # Abundances for the uxclumpy model

    m = model.Model("constant*phabs(atable{uxclumpy-cutoff.fits} + constant*atable{uxclumpy-cutoff-omni.fits} + mekal)")

    m(2).values = [nh_gal, -1]  # ISM absorption
    m(5).values = [400, -1]  # E-cutoff
    # m(6).values = [45, -1]  # TORsigma
    # m(7).values = [0.4, -1]  # CTKcover
    # m(8).values = [90.0, -1]  # Theta_inc
    m(9).values = [z, -1, 0.0, 0.0, 10.0, 10.0]  # Redshift
    m(11).values = [1.0e-5, 1.0e-5, 1.0e-5, 1.0e-5, 0.1, 0.1]
    m(21).values = [1.0, -1]  # nH for soft excess
    m(23).values = [z, -1]  # redshift

    if random_params:
        m(6).values = [random_params["torsigma"], -1]  # TORsigma
        m(7).values = [random_params["ctkcover"], -1]  # CTKcover
        m(8).values = [random_params["theta_inc"], -1]  # Theta_inc
        m(3).values = [10**(random_params["nh"]), -1]  # nH
        #m(9).values = [random_params["z"], -1, 0.0, 0.0, 10.0, 10.0]  # Redshift
        m(4).values = [random_params["phoindex"], -1]  # Photonindex

    ######## omni component
    if AllData.nGroups == 0:
        n_spec = 1
    else:
        n_spec = AllData.nGroups
    for i_model in range(1, n_spec + 1):
        AllModels(i_model)(12).link = m(3)
        AllModels(i_model)(13).link = m(4)
        AllModels(i_model)(14).link = m(5)
        AllModels(i_model)(15).link = m(6)
        AllModels(i_model)(16).link = m(7)
        AllModels(i_model)(17).link = m(8)
        AllModels(i_model)(18).link = m(9)
        AllModels(i_model)(19).link = m(10)

    set_pars(m, pars, par_vals, par_fixed, one_epoch, par_var)

    m(pars[0]).values = par_vals[0]
    m(1).values = 1.0, -1

    return m

In [13]:
def prepare_fakeit_settings(insts,exp, i):
    fk_s = []
    for i_i, inst in enumerate(insts):
        pha_path = save_path+"/spec_files/{}_{}.pha".format(inst,i)
        if os.path.exists(pha_path):
            os.remove(pha_path)
        fk_s.append(
            FakeitSettings(
                response=f"simulations/inst_files/{inst}.rmf",
                arf=f"simulations/inst_files/{inst}.arf",
                background=f"simulations/inst_files/{inst}_bgd.pha",
                exposure=exp[i_i] * 1000,
                fileName=pha_path
            )
        )
    return fk_s

In [14]:
def run_simulation(insts, logFs, exp, trials, start_ind=0):

    parameters = parameter_generator(trials, start_ind)

    #np.save("simulations/parameters.log",parameters)
    xss, xerrss, yss, yerrss = [], [], [], []
    
    for random_params in parameters:
        trial_id = random_params["trial_id"]
        print(f"Starting trial {trial_id} of {start_ind} ... {start_ind + trials} starting index with parameters: {random_params}")
        logNH = random_params["nh"]
        torsigma = random_params["torsigma"] # TORsigma
        ctkcover = random_params["ctkcover"]  # CTKcover
        theta_inc = random_params["theta_inc"]  # inclination angle
        phoindex = random_params["phoindex"]
            
        m = mk_clumpy(nh_gal, z, random_params)

        print(f"Trial {trial_id}, nH: {logNH}, Flux: {logFs}")
        norm = calculate_flux(logNH, logFs[0])
        m(10).values = norm
        fk_s = prepare_fakeit_settings(insts,  exp, trial_id)

        AllData.fakeit(len(insts), fk_s)

        AllData(1).ignore("1.0e-60-3.0")
        AllData(1).ignore("24.0-200.0")
        AllData(2).ignore("1.0e-60-3.0")
        AllData(2).ignore("24.0-200.0")
        AllData(3).ignore("1.0e-60-0.1")
        AllData(3).ignore("6.0-200.0")
        AllData(4).ignore("1.0e-60-0.1")
        AllData(4).ignore("6.0-200.0")
        AllData(5).ignore("1.0e-60-0.1")
        AllData(5).ignore("6.0-200.0")

        Plot.device = "/null"
        Plot.add = True
        # Fit.perform()

        Plot.xAxis="keV"
        Plot.background = True
        Plot.setRebin(10,20)

        Plot("ldata")
        xs = [Plot.x(i) for i in range(1,6)]
        xErrs = [Plot.xErr(i) for i in range(1,6)]
        ys = [Plot.y(i) for i in range(1,6)]
        yErrs = [Plot.yErr(i) for i in range(1,6)]
        xss.append(xs)
        xerrss.append(xErrs)
        yss.append(ys)
        yerrss.append(yErrs)
        AllData.clear()
    return xss, xerrss, yss, yerrss, parameters


In [ ]:
trials = 20
start_ind = 1

xss, xerrss, yss, yerrss, parameters=run_simulation(insts, logFs, exp, trials, start_ind)
savedata = {'E': xss, 
        'Eerr': xerrss,
        'rate': yss,
        'rateerr': yerrss,
        'pars': parameters}

np.save(save_path+f'/training/trials_{start_ind}_{start_ind+trials}.npy', savedata)

Starting trial 1 of 1 ... 21 starting index with parameters: {'trial_id': 1, 'torsigma': 21.114602443403783, 'ctkcover': 0.3910871692677794, 'nh': 24.925984389507665, 'theta_inc': 76.26493806327841, 'phoindex': 2.1522449465585156}
 Solar Abundance Vector set to wilm:  Wilms, J., Allen, A. & McCray, R. ApJ 542 914 (2000) (abundances are set to zero for those elements not included in the paper).

Model constant<1>*phabs<2>(atable{uxclumpy-cutoff.fits}<3> + constant<4>*atable{uxclumpy-cutoff-omni.fits}<5> + mekal<6>) Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   constant   factor              1.00000      +/-  0.0          
   2    2   phabs      nH         10^22    1.00000      +/-  0.0          
   3    3   torus      nH                  10.0000      +/-  0.0          
   4    3   torus      PhoIndex            2.00000      +/-  0.0          
   5    3   torus      Ecut                100.000      +/-  0.0          
   6    3   torus 

***Error: Desired Value 8.43304e+24 is outside hard range 0.01 - 10000


Trial 1, nH: 24.925984389507665, Flux: [-13]
 Model Flux 3.6494e-06 photons (4.5698e-14 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 3.6494e-06 photons (4.5698e-14 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 3.6494e-06 photons (4.5698e-14 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 3.6494e-06 photons (4.5698e-14 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 9.9893e-06 photons (1.2661e-13 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 7.9041e-06 photons (1e-13 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 7.9041e-06 photons (1e-13 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 8.0956e-06 photons (1.0244e-13 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 7.9041e-06 photons (1e-13 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 7.9041e-06 photons (1e-13 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 8.0956e-06 photons (1.0244e-13 ergs/cm^2/s) range (3.0000 - 24.000 keV)
 Model Flux 7.9041e-06 photons (1e-13 ergs/cm^2/s) 